# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [ ]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [ ]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [ ]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [ ]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [ ]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [ ]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [ ]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [ ]:
def jacobi(A, b, num_it=1000):
    diag_A = np.diag(A)    
    BB = -A.copy()             
    np.fill_diagonal(BB, 0)    
    D = np.diag(diag_A)        
    invD = np.diag(1./diag_A)
    B = invD @ BB             
    c = invD @ b
    norm = np.linalg.norm(B)
    
    x0 = np.ones(A.shape[0])
    x = x0
    for _ in range(num_it):
        x = B @ x + c
    
    return x, norm

In [ ]:
jacobi(A, b)

(array([ 0.03919429,  0.03780037,  0.04283232,  0.02365951,  0.05745031,
        -0.00030244, -0.00577279,  0.03177549, -0.00422849,  0.05284648]),
 0.36436161983015336)

In [ ]:
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15] * n)    # CHANGE THIS
b = rndm.uniform(size=n)

print(A)

[[1.51915195e+01 6.22108771e-01 4.37727739e-01 7.85358584e-01
  7.79975808e-01 2.72592605e-01 2.76464255e-01 8.01872178e-01
  9.58139354e-01 8.75932635e-01]
 [3.57817270e-01 1.55009951e+01 6.83462935e-01 7.12702027e-01
  3.70250755e-01 5.61196186e-01 5.03083165e-01 1.37684496e-02
  7.72826622e-01 8.82641191e-01]
 [3.64885984e-01 6.15396178e-01 1.50753812e+01 3.68824006e-01
  9.33140102e-01 6.51378143e-01 3.97202578e-01 7.88730143e-01
  3.16836122e-01 5.68098653e-01]
 [8.69127390e-01 4.36173424e-01 8.02147642e-01 1.51437668e+01
  7.04260971e-01 7.04581308e-01 2.18792106e-01 9.24867629e-01
  4.42140755e-01 9.09315959e-01]
 [5.98092228e-02 1.84287084e-01 4.73552788e-02 6.74880944e-01
  1.55946248e+01 5.33310163e-01 4.33240627e-02 5.61433080e-01
  3.29668446e-01 5.02966833e-01]
 [1.11894318e-01 6.07193706e-01 5.65944643e-01 6.76406199e-03
  6.17441709e-01 1.59121229e+01 7.90524133e-01 9.92081466e-01
  9.58801762e-01 7.91964135e-01]
 [2.85250960e-01 6.24916705e-01 4.78093796e-01 1.95675179e

In [ ]:
jacobi(A, b, 5000)

(array([ 0.03919429,  0.03780037,  0.04283232,  0.02365951,  0.05745031,
        -0.00030244, -0.00577279,  0.03177549, -0.00422849,  0.05284648]),
 0.36436161983015336)

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [ ]:
def seidel(A, b, n_iter=1000):
    diag_A = np.diag(A)        
    D = np.diag(diag_A)        
    invD = np.diag(1./diag_A)
    U = np.triu(A) - D
    L = np.tril(A) - D
    B_tilde = -np.linalg.inv(D+L) @ U 
    c_tilde = np.linalg.inv(D+L) @ b 
    
    norm = np.linalg.norm(B)
    
    x0 = np.ones(A.shape[0])
    x = x0
    
    for _ in range(n_iter):
        x = B_tilde @ x + c_tilde
    
    return x, norm

In [ ]:
np.set_printoptions(precision=3, suppress=True)

iters = range(1,15)

for i in iters:
    print('Jacobi', jacobi(A, b, i)[0])
    print('Seidel', seidel(A, b, i)[0])
    print('')

Jacobi [-0.332 -0.268 -0.279 -0.36  -0.126 -0.333 -0.208 -0.204 -0.342 -0.233]
Seidel [-0.332 -0.237 -0.196 -0.185 -0.052 -0.192 -0.051  0.064  0.02   0.096]

Jacobi [0.15  0.135 0.135 0.139 0.117 0.092 0.054 0.106 0.094 0.138]
Seidel [ 0.072  0.064  0.058  0.03   0.06  -0.006 -0.011  0.028 -0.008  0.051]

Jacobi [ 0.006  0.009  0.015 -0.011  0.04  -0.028 -0.024  0.01  -0.034  0.027]
Seidel [ 0.038  0.037  0.043  0.024  0.058  0.    -0.005  0.032 -0.004  0.053]

Jacobi [ 0.049  0.046  0.051  0.034  0.063  0.008 -0.     0.038  0.005  0.061]
Seidel [ 0.039  0.038  0.043  0.024  0.057 -0.    -0.006  0.032 -0.004  0.053]

Jacobi [ 0.036  0.035  0.04   0.021  0.056 -0.003 -0.007  0.03  -0.007  0.051]
Seidel [ 0.039  0.038  0.043  0.024  0.057 -0.    -0.006  0.032 -0.004  0.053]

Jacobi [ 0.04   0.039  0.044  0.025  0.058  0.    -0.005  0.032 -0.003  0.054]
Seidel [ 0.039  0.038  0.043  0.024  0.057 -0.    -0.006  0.032 -0.004  0.053]

Jacobi [ 0.039  0.038  0.043  0.023  0.057 -0.001 -0.006

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [ ]:
def minres(A, b, n_iter=500):
    x = np.ones(b.shape[0])
    T = []
    for _ in range(n_iter):
        r = A @ x - b
        tau = (r @ A @ r)/np.linalg.norm(A @ r)**2
        x = x - tau*r
        T.append(tau)
    return x, T

x = minres(A, b)[0]

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

In [ ]:
minres(A, b)[1]

[0.049458091915919704,
 0.06632815899696641,
 0.056465283719342306,
 0.05913686144833426,
 0.056149365850670686,
 0.05962056630692138,
 0.05532125962259336,
 0.06019139542967885,
 0.054726519624928915,
 0.060918354718292295,
 0.05478879426160098,
 0.061543172821534486,
 0.055177211118735095,
 0.06162125672274042,
 0.055145295102092665,
 0.0617011494677153,
 0.05472584159792917,
 0.062121454652208655,
 0.054002133608905656,
 0.0644186170983185,
 0.05443303564328217,
 0.06011645923749506,
 0.06271150857694172,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0